In [99]:
import gensim.downloader
import numpy as np
import pandas as pd
import pyarrow

In [100]:
glove_vectors = gensim.downloader.load("glove-wiki-gigaword-50")

In [101]:
data = pd.read_csv("../data/processed/abcnewsFinal.csv")

In [102]:
def toISO(data):
    data = str(data)
    return data[:4] + "-" + data[4:6] + "-" + data[6:]

In [103]:
def toVector(sentence):
    sentence = sentence.split(" ")
    currList = []
    for i in sentence:
        try:
            curr = glove_vectors[i.lower()]
            currList.append(curr)
        except:
            continue
    returnData = np.mean(currList, axis=0).tolist()
    if isinstance(returnData, list):
        return returnData
    return "delete"

In [104]:
data = data.rename(columns={"publish_date": "date", "headline_text": "headline"})
data["date"] = data.date.apply(toISO)
data["url"] = "null"
data

,date,headline,url
0,2003-02-19,aba decides against community broadcasting lic...,null
1,2003-02-19,act fire witnesses must be aware of defamation,null
2,2003-02-19,a g calls for infrastructure protection summit,null
3,2003-02-19,air nz staff in aust strike for pay rise,null
4,2003-02-19,air nz strike to affect australian travellers,null
...,...,...,...
1244177,2021-12-31,two aged care residents die as state records 2...,null
1244178,2021-12-31,victoria records 5;919 new cases and seven deaths,null
1244179,2021-12-31,wa delays adopting new close contact definition,null
1244180,2021-12-31,western ringtail possums found badly dehydrate...,null


In [105]:
data = data.iloc[:, [0, 2, 1]]
data

,date,url,headline
0,2003-02-19,null,aba decides against community broadcasting lic...
1,2003-02-19,null,act fire witnesses must be aware of defamation
2,2003-02-19,null,a g calls for infrastructure protection summit
3,2003-02-19,null,air nz staff in aust strike for pay rise
4,2003-02-19,null,air nz strike to affect australian travellers
...,...,...,...
1244177,2021-12-31,null,two aged care residents die as state records 2...
1244178,2021-12-31,null,victoria records 5;919 new cases and seven deaths
1244179,2021-12-31,null,wa delays adopting new close contact definition
1244180,2021-12-31,null,western ringtail possums found badly dehydrate...


In [106]:
data.to_parquet(
    "../data/processed/abcNewsWithoutEmbeds.parquet",
    engine="pyarrow",
    compression="snappy",
)

In [107]:
data["embedding"] = data.headline.apply(toVector)

/Users/kakhil2004/Desktop/School/Hacklytics/cLLiMate/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/var/folders/c2/0bns0k4961d6k4k62xsydvpc0000gn/T/ipykernel_13372/2658763695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["embedding"] = data.headline.apply(toVector)


In [108]:
data[data.embedding == "delete"]

,date,url,headline,embedding
904,2003-02-23,null,nataus,delete
2054,2003-02-28,null,natwa,delete
7219,2003-03-25,null,sptcri,delete
7220,2003-03-25,null,sptrby,delete
7635,2003-03-27,null,nattas,delete
...,...,...,...,...
1234951,2021-03-23,null,fcpsegment,delete
1237893,2021-06-28,null,hammersickleredflag,delete
1238521,2021-07-15,null,nomadland,delete
1241424,2021-10-11,null,4cs,delete


In [109]:
data = data[data.embedding != "delete"]

In [110]:
data

,date,url,headline,embedding
0,2003-02-19,null,aba decides against community broadcasting lic...,"[-0.5209766626358032, -0.07693099975585938, -0..."
1,2003-02-19,null,act fire witnesses must be aware of defamation,"[0.40227997303009033, -0.2763712406158447, -0...."
2,2003-02-19,null,a g calls for infrastructure protection summit,"[0.42550426721572876, 0.5782315135002136, 0.09..."
3,2003-02-19,null,air nz staff in aust strike for pay rise,"[0.11939321458339691, -0.014739777892827988, 0..."
4,2003-02-19,null,air nz strike to affect australian travellers,"[0.11280427873134613, -0.21327228844165802, 0...."
...,...,...,...,...
1244177,2021-12-31,null,two aged care residents die as state records 2...,"[0.10242997854948044, 0.2142380177974701, 0.23..."
1244178,2021-12-31,null,victoria records 5;919 new cases and seven deaths,"[0.2598228454589844, 0.307260125875473, 0.0266..."
1244179,2021-12-31,null,wa delays adopting new close contact definition,"[0.17442427575588226, 0.01617715135216713, -0...."
1244180,2021-12-31,null,western ringtail possums found badly dehydrate...,"[0.015956249088048935, -0.6479212641716003, -0..."


In [111]:
data.to_parquet(
    "../data/processed/abcNewsWithEmbeds.parquet",
    engine="pyarrow",
    compression="snappy",
)